In [13]:
import numpy as np # arrays & loading data
import tensorflow as tf # arrays & loading data
from tensorflow.keras.models import Sequential  # model type that we will use
from tensorflow.keras.layers import Dense # we will use Dense layers

# suppress warnings
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

In [55]:
# unpickle the data from the batch file in the CIFAR-10 dataset
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
# save the data as a dictionary
dict = unpickle("cifar-10-batches-py\\data_batch_1")

# split the data using the keys
labels = np.array(pickle[b'labels'])
data = np.array(pickle[b'data'])

# form a column vector where only 0 and 1 are kept ( we need binary classification)
labels_zero_or_one = np.where((labels<2),labels,-1)
labels_zero_or_one = labels_zero_or_one.reshape(-1,1) 

# concatanate the labels column form above to the data
data_concatanated = np.concatenate((data,labels_zero_or_one),axis=1)

# keep only the rows that have 0 or 1 as labels (those with -1 as labels are not important now)
data_final= data_concatanated[data_concatanated[:,-1]!=-1]
print(data_final)
print(data_final.shape)

[[170 168 177 ...  78  80   1]
 [159 150 153 ...  17  19   1]
 [202 202 204 ... 243 243   0]
 ...
 [116 120 126 ...  84  81   1]
 [ 71  60  74 ...  69  68   1]
 [250 254 211 ... 255 254   1]]
(1979, 3073)


In [56]:
X = data_final[:,:-1] # forming the input and output 
y = data_final[:,-1]

y = np.expand_dims(y, axis=1) # make y 2D - the commands later will require it


In [61]:
# split the data into TRAINING, CROSS-VALIDATION and TEST sets
from sklearn.model_selection import train_test_split

# TRAINING SET - 60%
X_train, X_temporary, y_train, y_temporary = train_test_split(X, y, test_size=0.40, random_state=1)

# the rest of 40% - CV SET(20%) and TEST SET(20%)
X_cv, X_test, y_cv, y_test = train_test_split(X_temporary, y_temporary, test_size=0.50, random_state=1)
del X_temporary, y_temporary

print(f"training input shape:{X_train.shape}")
print(f"training output shape:{y_train.shape}")
print(f"cv input shape:{X_cv.shape}")
print(f"cv output shape:{y_cv.shape}")
print(f"test input shape:{X_test.shape}")
print(f"test output shape:{y_test.shape}")

training input shape:(1187, 3072)
training output shape:(1187, 1)
cv input shape:(396, 3072)
cv output shape:(396, 1)
test input shape:(396, 3072)
test output shape:(396, 1)


In [85]:
# each image has 3072 units (this is explained in more detail in the documentation in the link provided)
# sequential model structure

model= Sequential(
    [
        tf.keras.Input(shape=(3072,)), # input size
        Dense(200,activation="sigmoid", name="layer1"),
        Dense(120,activation="sigmoid", name="layer2"),
        Dense(60,activation="sigmoid", name="layer3"),
        Dense(15,activation="sigmoid", name="layer4"),
        Dense(1,activation="sigmoid", name="layer5"),
    ], name="binary_model"
)

In [86]:
# see details about the activation of every layer and the form of the w and b parameters
model.summary()

Model: "binary_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                       │ (None, 200)                 │         614,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (Dense)                       │ (None, 120)                 │          24,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ (None, 60)                  │           7,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer4 (Dense)                       │ (None, 15)                  │             915 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer5 (Dense)                       │ (None, 1)                   │              16 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 646,911 (2.47 MB)

 Trainable params: 646,911 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# define loss and optimizer of the Adam's algorithm

model.compile(
    # this is similar to gradient descent, but it is a much improved version
    loss=tf.keras.losses.BinaryCrossentropy(), # BC - binary class & from_logits helps our calculations be more
    optimizer=tf.keras.optimizers.Adam(0.001), # preimplemented optimizer
)

In [88]:
# train the model "epochs" times
model.fit(
    X_train, y_train,
    epochs = 50
)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7046  
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6937
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6949
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6953
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6969
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6989
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6954
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6929
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6952
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6932
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6932
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6934
Epoch 13/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6947
Epoch 14/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6933
Epoch 15/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6933
Epoch 16/50
38/38